# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [46]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [47]:
weather_df = "output_data/my_weather_1.csv"

weather_data = pd.read_csv(weather_df)
weather_api = pd.DataFrame(weather_data)
weather_api.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Longyearbyen,78.2186,15.6401,-0.09,92,100,2.91,SJ,1584673200
1,1,Sioux Lookout,50.1001,-91.9170,16.43,92,100,13.06,CA,1584673200
2,2,Sapouy,11.5544,-1.7736,85.91,45,100,3.62,BF,1584673200
3,3,Buchanan,5.8808,-10.0467,77.45,89,70,3.18,LR,1584673200
4,4,Edd,13.9297,41.6925,75.83,87,100,10.47,ER,1584673200


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [48]:
gmaps.configure(api_key=g_key)
locations = weather_api[["Latitude", "Longitude"]].astype(float)
hum_rate = weather_api["Humidity"].astype(float)

In [49]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [37]:
city_df = weather_api[(weather_api['Max Temp'] < 80) 
                        & (weather_api['Max Temp'] > 70) 
                        & (weather_api['Wind Speed'] < 10) 
                        & (weather_api['Cloudiness'] == 0)]
city_df = narrowed_city_df.dropna(how='any')
city_df.reset_index(inplace=True)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = city_df

params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
name_address = requests.get(url, params=params)

In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [30]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))